Example server with Flask demonstrating use of OAuth 2.0. Server<br>
needs to be deployed. Example code is requesting access token from<br>
Bitbucket. User has to grant access rights. After authorization the<br>
token and the available workspaces are returned.

In [ ]:
from requests_oauthlib import OAuth2Session
from atlassian.bitbucket import Cloud
from flask import Flask, request, redirect, session

In [ ]:
app = Flask(__name__)
app.secret_key = ""

Bitbucket OAuth URLs

In [ ]:
authorization_base_url = "https://bitbucket.org/site/oauth2/authorize"
token_url = "https://bitbucket.org/site/oauth2/access_token"

1. Create OAuth consumer<br>
Go to "Your profile and setting" -> "All workspaces" -> "Manage".<br>
Go to "Apps and features" -> "OAuth consumers".<br>
Click "Add consumer". Fill in Name and Callback URL. Set permissions<br>
as needed. Click save and copy client id and secret.

In [ ]:
client_id = ""
client_secret = ""

2. Redirect to Bitbucket for authorization<br>
The server request to {server_url}/login is redirected to Bitbucket.<br>
The user is asked to grant access permissions.

In [ ]:
@app.route("/login")
def login():
    bitbucket = OAuth2Session(client_id)
    authorization_url, state = bitbucket.authorization_url(authorization_base_url)
    session["oauth_state"] = state
    return redirect(authorization_url)

3. Bitbucket sends callback with token<br>
Bitbucket is calling the Callback URL specified in the OAuth<br>
consumer. This should be set to {server_url}/callback. The callback<br>
contains the access token.

In [ ]:
@app.route("/callback")
def callback():
    bitbucket = OAuth2Session(client_id, state=session["oauth_state"])
    token = bitbucket.fetch_token(token_url, client_secret=client_secret, authorization_response=request.url)
    return "Token: {}<p />Workspaces: {}".format(token, ", ".join(get_workspaces(token)))

4. Token used for Bitbucket Python API<br>
Bitbucket Cloud API library is called with token information. User is<br>
authenticated with OAuth 2.0.